# Fourier Transform Based Outlier Detection in Time Series data using CuPy and CLX libraries

## Authors:
   Gorkem Batmaz (Nvidia)

## Contents

* Introduction
* Implementation using CuPy and CLX
* References

## Introduction

Outlier detection is a  technique for detecting outliers in the observed data or in time series. In real life, signals of interest can have many sinusoidal components and a lot of noise. Therefore it is not always straightforward to create rules to catch anomalies. We will demonstrate how we can detect outliers in time series data using Fourier transform.
We quote from -Brighan, 1974, The FFT- "..., the essence of the Fourier Transform of a waveform is to decompose or separate the waveform into a sum of sinusoids of different frequencies. If these sinusoids sum to the original waveform then we have determined the Fourier Transform of the waveform." Mathematically, this transform is stated as:

$X(\omega)=\int_{-\infty}^{\infty} x(t) e^{-t \omega t} d t$

We will use discrete Fourier transform for performing Fourier analysis of discrete-time sequences. It will help us denoise the signal and allow us to employ statistical methods to detect anomalies.

The Discrete Fourier Transform $ X[k] $ of $ x[n] $ is the discrete-time signal is  

$ X[k]=\sum_{k=0}^{N−1} x[n].e^{−J.2pi.kn/N} $

where $ x[n] $ is a discrete-time signal with Period N.

The Inverse Discrete Fourier transform is

$ x[n]=(1/N)\sum_{k=0}^{N−1} X[k].e^{J.2pi.kn/N} $

To do this, we will first move the signal into the frequency domain. Then we will apply kind of a filter to remove components that have too high frequencies. Once we have a more precise signal, it will be possible to use techniques such as rolling z score to detect anomalies.
In this notebook, we will use an alternative dataset not to disclose internal information. We will use a dataset of daily minimum temperature measurements taken in Melbourne, Australia between 1981 and 1990. This dataset is available to download at https://www.kaggle.com/paulbrabban/daily-minimum-temperatures-in-melbourne

### Implementation

#### Importing the libraries

In [1]:
import cupy as cp
import cudf
# try:
#     import clx 
# except ImportError:
#     !conda install -c rapidsai -y clx
import clx 
from clx.analytics.stats import rzscore
import numpy as np
try:
    import matplotlib as pyplot
except ImportError:
    !conda install -c rapidsai -y matplotlib
import matplotlib.pyplot as pyplot

# try:
#     import cuxfilter
# except ModuleNotFoundError:
#     os.system('conda install -c rapidsai -y cuxfilter')   
import cuxfilter
from cuxfilter.charts import scatter, bokeh, line
from cuxfilter import DataFrame, themes, layouts
from cuxfilter.assets.custom_tiles import get_provider, Vendors
from bokeh.palettes import Magma, Inferno, Plasma, Viridis, Cividis, Category20c

IndentationError: unexpected indent (<ipython-input-1-e660ba37fa40>, line 7)

#### Read the data into GPU memory

In [ ]:
gdf=cudf.read_csv("daily-min-temperatures.txt")

In [ ]:
gdf['Temp']

#### Convert cudf series into CuPy to utilise the Fast Fourier Transform functions

In [ ]:
a = cp.fromDlpack(gdf['Temp'].to_dlpack())

In [ ]:
a

#### Visualise the data

In [ ]:
pyplot.plot(cp.asnumpy(a))

We can also now visualize our data using the `cuxfilter` library.

In [ ]:
reshaped_df = cudf.DataFrame(a)
reshaped_df = reshaped_df.reset_index()
print(reshaped_df)

In [ ]:
reshaped_df.columns =['x' , 'y']
print(reshaped_df)

In [ ]:
demo_red_blue_palette = [ "#3182bd", "#6baed6", "#7b8ed8", "#e26798", "#ff0068" , "#323232" ]

cux_df = DataFrame.from_dataframe(reshaped_df)

line_chart_1 = line(x='x',y='y')

d = cux_df.dashboard([line_chart_1], title="")
line_chart_1.view()

The raw signal seems to have multiple components, yet it's noisy, and it's not straightforward to detect any outliers.

#### Transfer the data to the Frequency domain

There is a very efficient algorithm to calculate the DFT called FFT. We quote from -MULGREW, 1997, DSP- "The FFT is based on the observation that there are many symmetries in the DFT matrix and that many multiplications were being needlessly repeated."  The equation changes into: 

$\begin{aligned} X(k) &=\sum_{n=0}^{N-1} x(n) W_{M}^{k n}, \quad k=0,1, \ldots, N-1 \\ &=\sum_{n \text { even }} x(n) W_{N}^{k n}+\sum_{n \text { odd }} x(n) W_{M}^{k x} \\ &=\sum_{m=0}^{(M / 2)-1} x(2 m) W_{M}^{2 m k}+\sum_{m=0}^{(M / 2)-1} x(2 m+1) W_{M}^{k(2 m+1)} \end{aligned}$

Because we are computing the one dimensional discrete Fourier Transform of a real-valued signal, we can use a function called `rfft` in CuPy 

In [ ]:
rft= cp.fft.rfft(a)

Setting the first 15 frequencies to zero was enough to clean our signal when we applied the inverse the Fast Fourier Transform. This number needs to be adjusted with other datasets.

In [ ]:
freq_num=15

In [ ]:
rft[freq_num:] = 0
y_smooth = cp.fft.irfft(rft)

In [ ]:
y_smooth

In [ ]:
pyplot.plot(cp.asnumpy(y_smooth))

We can also now visualize our data using the `cuxfilter` library.

In [ ]:
reshaped_df = cudf.DataFrame(y_smooth)
reshaped_df = reshaped_df.reset_index()
print(reshaped_df)

In [ ]:
reshaped_df.columns =['x' , 'y']
print(reshaped_df)

In [ ]:
cux_df = DataFrame.from_dataframe(reshaped_df)

line_chart_2 = line(x='x',y='y')

d = cux_df.dashboard([line_chart_2])
line_chart_2.view()

In [ ]:
a

We calculate the difference between the new smooth signal and the original.

In [ ]:
err=(abs(y_smooth-a))

Convert the data into a cudf series so that we can use the clx analytics library to calculate the rolling Z-score

We select rolling window as 5. This should be adjusted for other datasets.

In [ ]:
rwindow=5

In [ ]:
rz=rzscore(cudf.Series(cp.ascontiguousarray((err))),rwindow)[rwindow-1:] #Remove the NA rows

In [ ]:
rz

In [ ]:
pyplot.plot(cp.asnumpy(cp.asarray(rz)))

We can now visualize our data using the `cuxfilter` library.

In [ ]:
reshaped_df = cudf.DataFrame(cp.asarray(rz))
reshaped_df = reshaped_df.reset_index()
print(reshaped_df)

In [ ]:
reshaped_df.columns =['x' , 'y']
print(reshaped_df)

In [ ]:
cux_df = DataFrame.from_dataframe(reshaped_df)

line_chart_3 = line(x='x',y='y')

d = cux_df.dashboard([line_chart_3])
line_chart_3.view()

In this plot, we can see that the rolling Z-scores mostly vary between -2 and 2. This view may help us select a threshold.

Convert the data back into a CuPy array

In [ ]:
rzn=cp.fromDlpack(rz.to_dlpack())

In [ ]:
rzn

#### Select a rolling z score threshold

This threshold should be adjusted according to the nature of each dataset. For this dataset, we will select the rolling z-score threshold as 1.99, and will mark any value that exceeds this as an anomaly. 

In [ ]:
rz_threshold=1.99

In [ ]:
len(rzn[abs(rzn)>rz_threshold])

Indices of the outliers:

In [ ]:
cp.argwhere(abs(rzn)>1.99)

## Conclusion

We have identified two outliers when rolling z-score threshold is selected 1.99 and returned the indices of the outliers. This work can be expanded as a monitoring tool for time-series data in the cyber domain and to highlight the anomalies.

## References

##### Rasheed, Peng, Alhajj, Rokne, Jon: Fourier Transform Based Spatial Outlier Mining, 2009
##### BRIGHAN, 1974, The Fast Fourier Transform
##### MULGREW, GRANT, THOMPSON, 1997, Digital Signal Processing
##### OPPENHEIM, 1997, Signals & Systems